### Rerank  
  
- 주어진 순위를 재정렬하여 더 나은 순위를 제공하는 작업  
- 정보 검색, 추천 시스템 등 순위가 영향을 미치는 영역에서 사용  
- 기본 아이디어는 초기 순위를 생성한 후에 추가적인 특성이나 기준을 고려하여 더 나은 순위를 만드는 것  
실시간 ?? 예시가 기억이 안난당

#### Rerank 알고리즘의 종류  
1. Learning to Rank(LTR)  
- 기계 학습 기술을 사용하여 초기 순위를 향상시키는 방법  
- RankNet, LambdaMART, RankBoost 등이 존재  
- 특징을 추출하고 학습하여 더 나은 순위를 생성  
  
2. Relevance Feedback  
- 사용자 피드백을 기반으로 초기 순위를 개선
- 사용자가 선택한 항목에 대한 피드백을 수집하여 해당 정보를 활용하여 순위 조정  
  
3. Query Expansion  
- 초기 검색 쿼리를 확장하여 다양한 관련 용어를 포함하여 초기 순위를 개선
- 검색 쿼리를 향상시켜 더 많은 관련 문서를 찾는데 도움을 줌  
  
4. Diversification  
- 다양성 증가시켜 사용자가 다양한 관점에서 정보를 확인할 수 있도록 하는 방법  
  
5. Temporal Reranking  
- 시간적 특성을 고려하여 검색 겨로가를 재정렬하는 방법  
- 실시간 이벤트나 트렌드에 관련된 검색에 유용  
  
6. User Context-based Reranking  
- 사용자 정보 기반 초기 순위 개선  
- 사용자 위치, 장치, 이전 검색 기록을 고려하여 더 관련성 높은 결과를 제공  
  
7. Combination Approaches 
- 여러 reranking 전략을 결합하여 더 나은 결과를 달성하는 방법  
  



### Rerank 기본 코드

In [124]:
import numpy as np

# 초기 순위를 가정한 데이터 (예: 검색 결과)
initial_ranking = [
    {'document_id': 1, 'score': 4},
    {'document_id': 2, 'score': 3},
    {'document_id': 3, 'score': 3.5},
    {'document_id': 4, 'score': 2.5},
    {'document_id': 5, 'score': 4.5}
]

# 추가 특성 데이터 (예: 문서의 길이)
additional_features = {
    1: {'length': 100},
    2: {'length': 80},
    3: {'length': 120},
    4: {'length': 90},
    5: {'length': 110}
}

# reranking을 위한 함수
def rerank(initial_ranking, additional_features):
    '''
    문서의 길이와 스코어를 조합하여 리랭킹
    문서의 길이와 스코어를 조합한 결과로 소팅할 것(높은 순)    
    '''    
    reranked_list = []

    lst = []
    for i in range(len(initial_ranking)):
        lst.append(initial_ranking[i]['score']*additional_features[i+1]['length']) # 스코어 * 길이 값 lst에 저장

    df = pd.DataFrame(lst, index=[i+1 for i in range(5)])
    df = df.sort_values(0, ascending=False)
    i_lst = list(df.index)
    new_score_lst = []
    for i in range(len(initial_ranking)): # new_score_dct 생성
        new_score_lst.append({'document_id': i+1, 'score': lst[i]})
    for i in range(len(new_score_lst)):
        reranked_list.append(new_score_lst[i_lst[i]-1])
    return reranked_list

# reranking 수행
reranked_results = rerank(initial_ranking, additional_features)

# 결과 출력
print("Reranked Results:")
for item in reranked_results:
    print(f"Document ID: {item['document_id']}, Rerank Score: {item['score']}")

Reranked Results:
Document ID: 5, Rerank Score: 495.0
Document ID: 3, Rerank Score: 420.0
Document ID: 1, Rerank Score: 400
Document ID: 2, Rerank Score: 240
Document ID: 4, Rerank Score: 225.0


In [119]:
# claude 답
import numpy as np

# 초기 순위를 가정한 데이터 (예: 검색 결과)
initial_ranking = [
    {'document_id': 1, 'score': 4},
    {'document_id': 2, 'score': 3},
    {'document_id': 3, 'score': 3.5},
    {'document_id': 4, 'score': 2.5},
    {'document_id': 5, 'score': 4.5}
]

# 추가 특성 데이터 (예: 문서의 길이)
additional_features = {
    1: {'length': 100},
    2: {'length': 80},
    3: {'length': 120},
    4: {'length': 90},
    5: {'length': 110}
}

def rerank(initial_ranking, additional_features):
    
    # 스코어와 길이를 조합하여 새로운 점수 계산
    reranked_scores = [item['score'] * additional_features[item['document_id']]['length'] for item in initial_ranking]

    # 새로운 점수를 기준으로 initial_ranking을 정렬.
    # zip을 sorted 하면 첫번째 요소를 기준으로 정렬
    reranked_results = [item for _, item in sorted(zip(reranked_scores, initial_ranking), reverse=True)]
    reranked_scores.sort(reverse=True)

    # 정렬된 리스트에 새로운 점수 추가
    for i, item in enumerate(reranked_results):
        item['score'] = reranked_scores[i]
    
    return reranked_results

# reranking 수행
reranked_results = rerank(initial_ranking, additional_features)

# 결과 출력
print("Reranked Results:")
for item in reranked_results:
    print(f"Document ID: {item['document_id']}, Rerank Score: {item['score']}")

Reranked Results:
Document ID: 5, Rerank Score: 495.0
Document ID: 3, Rerank Score: 420.0
Document ID: 1, Rerank Score: 400
Document ID: 2, Rerank Score: 240
Document ID: 4, Rerank Score: 225.0


In [123]:
# gpt 답

initial_ranking = [
    {'document_id': 1, 'score': 4},
    {'document_id': 2, 'score': 3},
    {'document_id': 3, 'score': 3.5},
    {'document_id': 4, 'score': 2.5},
    {'document_id': 5, 'score': 4.5}
]

additional_features = {
    1: {'length': 100},
    2: {'length': 80},
    3: {'length': 120},
    4: {'length': 90},
    5: {'length': 110}
}

# reranking을 위한 함수
def rerank(initial_ranking, additional_features):
    # 문서의 길이와 스코어를 조합하여 리랭킹
    reranked_list = []

    for doc in initial_ranking:
        doc_id = doc['document_id']
        score = doc['score']
        length = additional_features[doc_id]['length']
        combined_score = score * length  # 스코어와 길이를 곱하여 새 스코어 계산
        reranked_list.append({'document_id': doc_id, 'score': combined_score})

    # 새로운 스코어(rerank_score)로 내림차순 정렬
    reranked_list.sort(key=lambda x: x['score'], reverse=True)

    return reranked_list

reranked_results = rerank(initial_ranking, additional_features)

print("Reranked Results:")
for item in reranked_results:
    print(f"Document ID: {item['document_id']}, Rerank Score: {item['score']}")

Reranked Results:
Document ID: 5, Rerank Score: 495.0
Document ID: 3, Rerank Score: 420.0
Document ID: 1, Rerank Score: 400
Document ID: 2, Rerank Score: 240
Document ID: 4, Rerank Score: 225.0


### RankNet

- 주어진 문서들을 비교하여 문서들의 특성을 습득  
- 새로 주어진 문서는 어떤 문서와 유사한지 예측하는 알고리즘  

#### 작동 원리  
1. Input Representation  
- 각 문서는 특징 벡터로 표현  
- 특징 벡터: 문서의 속성으로 TF-IDF 혹은 문서 길이, 종류 등  
  
2. Pairwise Comparison  
- 주어진 문서들을 비교하여 상대적인 순서를 학습  
  
3. Training  
- 신경망을 통해 학습
- 예측된 순서와 실제 순서 간의 차이를 최소화하는 방법으로 가중치를 조정  
  
4. Ranking
- 학습된 RankNet 모델을 사용하여 새로운 문서들의 순위를 예측  
- 각 문서에 대한 점수를 출력하고 이를 기반으로 최종 순위를 생성

In [66]:
# tensorflow
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
# 간단한 데이터셋 생성
# 각 문서는 3개의 특징을 가짐
# 두 문서 쌍의 순서가 주어짐 (1: 첫 번째 문서가 더 관련성이 높음, 0: 두 번째 문서가 더 관련성이 높음)
X_train = np.array([[0.1, 0.2, 0.3], [0.3, 0.2, 0.5]])
y_train = np.array([1, 0])

# RankNet 모델 정의
model = tf.keras.Sequential()
'''
첫번째 dense layer hidden_size: 64, activation: relu
두번째 dense layer hidden_size: 32, activation: relu
세번째 dense layer hidden_size: 1, activation: sigmoid
'''
# )
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 컴파일
'''
 2개이므로 2개에 맞는 로스함수 설정
 옵티마이저: 아담
 메트릭: 정확도
'''
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='binary_crossentropy',
             metrics=['accuracy'])
# 모델 학습
'''
 에폭 10
'''
model.fit(X_train, y_train, epochs=10)

# 학습된 모델을 사용하여 순위 예측
X_test = np.array([[0.2, 0.3, 0.4]])
predictions = model.predict(X_test)
print("Predicted ranking:", predictions)

Epoch 1/10
1/1 [==============================] - 1s 917ms/step - loss: 0.6969 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6945 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 6ms/step - loss: 0.6914 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 7ms/step - loss: 0.6899 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6883 - accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6869 - accuracy: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6857 - accuracy: 0.5000
Epoch 9/10
1/1 [==============================] - 0s 6ms/step - loss: 0.6846 - accuracy: 0.5000
Epoch 10/10
1/1 [==============================] - 0s 81ms/step
Predicted ranking: [[0.49739432]]


In [62]:
# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# RankNet 모델 정의
class RankNet(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2):
        super(RankNet, self).__init__()


    def forward(self, x):
        return x

# 데이터셋 생성
# 각 문서는 3개의 특징을 가짐
# 두 문서 쌍의 순서가 주어짐 (1: 첫 번째 문서가 더 관련성이 높음, 0: 두 번째 문서가 더 관련성이 높음)
X_train = torch.tensor([[0.1, 0.2, 0.3], [0.3, 0.2, 0.5]], dtype=torch.float32)
y_train = torch.tensor([1, 0], dtype=torch.float32).view(-1, 1)

# RankNet 모델 초기화
input_size = torch.zeros(X_train.shape)
hidden_size = torch.zeros(8,8)
hidden_size2 = ''
model = RankNet(input_size, hidden_size, hidden_size2)

# Loss 및 Optimizer 정의
criterion = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
optimizer = ''

# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    '''
    '''
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 학습된 모델을 사용하여 순위 예측
X_test = torch.tensor([[0.2, 0.3, 0.4]], dtype=torch.float32)
predictions = model(X_test)
print("Predicted ranking:", predictions.item())

NameError: name 'loss' is not defined

In [63]:
input_size

tensor([[0., 0., 0.],
        [0., 0., 0.]])